In [98]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
import string
import json
import re

In [99]:
pd.set_option('display.max_colwidth', 500)

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mok\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mok\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

## Import file
- Only keep rows with non-null titles and abstracts

In [3]:
m = pd.read_csv('./CORD-19-research-challenge/metadata.csv')

C:\Users\mok\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
m = m[(m['title'].notna() & m['abstract'].notna())]

## Lemmatize Abstracts and Titles (get word roots)

- lowercase
- remove punctuation
- remove stopwords
- get root of words

In [5]:
w_tokenizer = WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

In [6]:
def preprocess(sentence):
    sentence = sentence.lower()
    sentence_no_punctuation = sentence.translate(str.maketrans('', '', string.punctuation))
    lemmatized_list = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(sentence_no_punctuation) 
                  if w not in stopwords.words('english')]
    return ' '.join(word for word in lemmatized_list)

In [14]:
m['cleanTitle']=m['title'].map(lambda s:preprocess(s)) 

In [15]:
m['cleanAbstract']=m['abstract'].map(lambda s:preprocess(s)) 

In [16]:
#m.to_csv('./CORD-19-research-challenge/metadata_lemmatized.csv', index=False)

In [101]:
#m = pd.read_csv('./CORD-19-research-challenge/metadata_lemmatized.csv')

In [104]:
m.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'has_pdf_parse',
       'has_pmc_xml_parse', 'full_text_file', 'url', 'cleanTitle',
       'cleanAbstract'],
      dtype='object')

In [105]:
m.shape

(38899, 20)

In [129]:
date_filter = '2019-12-01'
m[m['publish_time']>date_filter].shape

(3628, 21)

## Keywords

- Need a core covid keyword
- And need a topic keyword

## Make sure keywords are lemmatized version

In [18]:
check_words = ['isolation',
                'social',
                'distancing',
                'contact',
                'tracing',
                'event',
                'cancelation',
                'case',
                'isolation',
                'shelter',
                'place', 
                'stay',
                'home', 
                'movement',
                'restriction',
                'economic', 
                'unemployment',
                'depression',
                'financial',
                'crisis',
                'market',
                'stock',
                'macroeconomics',
                'dsge',
                'face',
                'facial',
                'mask',
                'travel',
                'ban',
                'school',
                'closure'] 

In [19]:
[lemmatizer.lemmatize(w) for w in check_words if lemmatizer.lemmatize(w) != w]

[]

## Confirmation all words at root word

In [20]:
covid_keywords = ['corona', 'covid']

In [21]:
intervention_keywords = [#'isolation', #can't use - too broad 
         # i.e. isolation of sequences from a random-sequence expression library that mimic viral epitopes
                        'social distancing',
                        'contact tracing',
                        'case isolation',
                        'shelter place', # in/at is removed stopword
                        'stay home', # at is removed stopword
                        'movement restriction',
                        'event cancellation',
                        'face mask',
                        'facial mask',
                        'travel ban',
                        'school closure']

In [22]:
economic_keywords = ['economic', 
                    'unemployment',
                     'unemploy',
                   # 'depression', # mental health articles here
                # i.e. 148. acute bipolar disorder depression is associated with immune activation
                    'financial crisis',
                  #  'market', # related to physical markets
            # i.e. wet markets—a continuing source of severe acute respiratory syndrome and influenza?
                #    ' stock', # often completly unrelated (or related to livestock if no space)
            # i.e. plaque assay for human coronavirus nl63 using human colon carcinoma cells
                    'stock market',
                    'macroeconomics',
                    'dsge']

## Create Functions

In [23]:
def find_papers_w_keywords(topic_keywords, papers):
    for keyword in topic_keywords:
        num_papers_title = len(papers[(papers['cleanTitle'].str.contains(keyword)) & 
                                        (papers['cleanTitle'])])
        num_papers_abstract = len(papers[papers['cleanAbstract'].str.contains(keyword)])
        print ('Identified {} papers with "{}" in title, {} relevant papers with "{}" in abstract'\
                       .format(num_papers_title, keyword, num_papers_abstract, keyword)) 

In [106]:
# 0 for title search, 1 for abstract search
def return_papers(topic_keywords, papers, abstract_search=1):
    relevant_papers = pd.DataFrame(columns=['title', 'abstract', 'sha', 'full_text_file', 'has_pdf_parse', 'has_pmc_xml_parse', 'publish_time'])
    if abstract_search == 1:
        for keyword in topic_keywords:
            relevant_papers = pd.concat((relevant_papers, papers[['title', 'abstract', 
                                'sha', 'full_text_file', 'has_pdf_parse', 'has_pmc_xml_parse', 'publish_time']][papers['cleanAbstract'].str.contains(keyword)]))
    else:
        for keyword in topic_keywords:
            relevant_papers = pd.concat((relevant_papers, papers[['title', 'abstract', 
                        'sha', 'full_text_file', 'has_pdf_parse', 'has_pmc_xml_parse', 'publish_time']][papers['cleanTitle'].str.contains(keyword)]))
    
    return relevant_papers.drop_duplicates()

## Identify core papers

In [107]:
find_papers_w_keywords(covid_keywords, m)

Identified 5207 papers with "corona" in title, 8427 relevant papers with "corona" in abstract
Identified 1124 papers with "covid" in title, 1547 relevant papers with "covid" in abstract


In [108]:
m['core_abstract'] = m['cleanAbstract'].apply(lambda x: any([k in x for k in covid_keywords]))

In [109]:
len(m), len(m[m['core_abstract'] == True])

(38899, 8965)

In [110]:
covid_papers = m[m['core_abstract'] == True]

## Intervention Papers


In [111]:
find_papers_w_keywords(intervention_keywords, covid_papers)

Identified 9 papers with "social distancing" in title, 73 relevant papers with "social distancing" in abstract
Identified 7 papers with "contact tracing" in title, 54 relevant papers with "contact tracing" in abstract
Identified 1 papers with "case isolation" in title, 7 relevant papers with "case isolation" in abstract
Identified 0 papers with "shelter place" in title, 2 relevant papers with "shelter place" in abstract
Identified 0 papers with "stay home" in title, 5 relevant papers with "stay home" in abstract
Identified 0 papers with "movement restriction" in title, 4 relevant papers with "movement restriction" in abstract
Identified 1 papers with "event cancellation" in title, 3 relevant papers with "event cancellation" in abstract
Identified 0 papers with "face mask" in title, 14 relevant papers with "face mask" in abstract
Identified 0 papers with "facial mask" in title, 1 relevant papers with "facial mask" in abstract
Identified 2 papers with "travel ban" in title, 14 relevant p

In [112]:
intervention_papers = return_papers(intervention_keywords, covid_papers, 1)

In [113]:
len(intervention_papers)

163

In [130]:
intervention_papers = intervention_papers[intervention_papers['publish_time']>date_filter]
len(intervention_papers)

140

In [131]:
intervention_papers.head()

,title,abstract,sha,full_text_file,has_pdf_parse,has_pmc_xml_parse,publish_time
2254,Coronavirus Disease 2019 (COVID-19) Pandemic and Pregnancy,"Abstract The current coronavirus disease 2019 (COVID-19) pneumonia pandemic, caused by the severe acute respiratory syndrome 2 (SARS-CoV-2) virus, is spreading globally at an accelerated rate, with a basic reproduction number (R0) of 2 – 2.5, indicating that 2 – 3 persons will be infected from an index patient. A serious public health emergency, it is particularly deadly in vulnerable populations and communities in which healthcare providers are insufficiently prepared to manage the infectio...",5dc4268a42adf3d5c55c87b7f6518de600b057c5,custom_license,True,False,2020-03-23
3836,COVID-19 and Italy: what next?,"Summary The spread of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) has already taken on pandemic proportions, affecting over 100 countries in a matter of weeks. A global response to prepare health systems worldwide is imperative. Although containment measures in China have reduced new cases by more than 90%, this reduction is not the case elsewhere, and Italy has been particularly affected. There is now grave concern regarding the Italian national health system's capacity to ...",849f0d4e93647e40b3a6f0841ebb2dd6a890a6b7,custom_license,True,False,2020-03-13
7308,Unknown unknowns – COVID-19 and potential global mortality,"Abstract COVID-19 (SARS-CoV-2) is currently a global pandemic. This paper will attempt to estimate global infection rates and potential resultant mortality in the absence of effective treatment and/or vaccination. Calculations are based on World Health Organisation data from Wuhan in China: 14% of infected cases are severe, 5% require intensive care and 4% die. Estimated infection rates and mortality rates at the level of continents and some individual countries (when these are of sufficient...",b039cc050d0d94bf4783a550e68f3a0ce2f1f796,custom_license,True,False,2020-03-31
7914,Using psychoneuroimmunity against COVID-19,"Abstract The worldwide outbreak of coronavirus disease 2019 (COVID-19) raises concerns of widespread panic and anxiety in individuals subjected to the real or perceived threat of the virus. Compared to general populations, patients who are institutionalized in a closed unit are also very vulnerable to COVID-19 infection and complications. This crisis touched on difficult issues of not only psychiatric care and ethics, but also psychological impacts to psychiatric care givers. In this Viewpoi...",4f47977f3202bca320c42ccbe3e0432c8bfab2f5,custom_license,True,False,2020-03-29
9429,Transmission potential and severity of COVID-19 in South Korea,"Abstract Objectives Since the first case of 2019 novel coronavirus (COVID-19) identified on Jan 20, 2020 in South Korea, the number of cases rapidly increased, resulting in 6,284 cases including 42 deaths as of March 6, 2020. To examine the growth rate of the outbreak, we aimed to present the first study to report the reproduction number of COVID-19 in South Korea. Methods The daily confirmed cases of COVID-19 in South Korea were extracted from publicly available sources. By using the empiri...",1aed8cc899c77553312d72e2ec022fd77e50b4fb,custom_license,True,False,2020-03-18


In [132]:
intervention_papers.to_csv("intervention_abstracts.csv", index=False)

## Economic Papers

In [133]:
find_papers_w_keywords(economic_keywords, covid_papers)

Identified 10 papers with "economic" in title, 290 relevant papers with "economic" in abstract
Identified 0 papers with "unemployment" in title, 0 relevant papers with "unemployment" in abstract
Identified 0 papers with "unemploy" in title, 0 relevant papers with "unemploy" in abstract
Identified 0 papers with "financial crisis" in title, 0 relevant papers with "financial crisis" in abstract
Identified 0 papers with "stock market" in title, 0 relevant papers with "stock market" in abstract
Identified 0 papers with "macroeconomics" in title, 1 relevant papers with "macroeconomics" in abstract
Identified 0 papers with "dsge" in title, 0 relevant papers with "dsge" in abstract


In [134]:
economic_papers = return_papers(economic_keywords, covid_papers, 1)

In [135]:
len(economic_papers)

290

In [136]:
economic_papers = economic_papers[economic_papers['publish_time']>date_filter]
len(economic_papers)

80

In [137]:
economic_papers[economic_papers['abstract'].str.contains('economic')]

,title,abstract,sha,full_text_file,has_pdf_parse,has_pmc_xml_parse,publish_time
1650,Human Coronaviruses: General Features,"Abstract Human coronaviruses (HCoVs), including HCoV-229E, HCoV-OC43, HCoV-NL63, and HCoV-HKU1, are traditionally known to cause symptoms of common cold with only moderate clinical impact. Severe acute respiratory syndrome coronavirus (SARS-CoV) and Middle East respiratory syndrome coronavirus (MERS-CoV), on the other hand, have strike humans in the past two decades as highly fatal human pathogens leading to considerable mortality and economic loss. This article summaries the updates on the ...",90798ce4da0c11f8eba3a943743b0a1584ff046a,custom_license,True,False,2019-12-31
1892,Chapter 3 Infectious Bronchitis Virus in Poultry: Molecular Epidemiology and Factors Leading to the Emergence and Reemergence of Novel Strains of Infectious Bronchitis Virus,"Abstract Infectious bronchitis virus (IBV) is a coronavirus that causes an acute and highly contagious disease in chickens. The virus can cause substantial economic losses throughout the poultry industry worldwide. It can affect the upper respiratory tract and the reproductive tract, and some strains can cause nephritis. The causative agent IBV is an RNA virus with great ability for mutation and recombination, thus capable of generating new virus strains that are difficult to control. There ...",a0de24c5fdb4a0e77b1b5e8c002ced81ebe9b227,custom_license,True,False,2020-12-31
2254,Coronavirus Disease 2019 (COVID-19) Pandemic and Pregnancy,"Abstract The current coronavirus disease 2019 (COVID-19) pneumonia pandemic, caused by the severe acute respiratory syndrome 2 (SARS-CoV-2) virus, is spreading globally at an accelerated rate, with a basic reproduction number (R0) of 2 – 2.5, indicating that 2 – 3 persons will be infected from an index patient. A serious public health emergency, it is particularly deadly in vulnerable populations and communities in which healthcare providers are insufficiently prepared to manage the infectio...",5dc4268a42adf3d5c55c87b7f6518de600b057c5,custom_license,True,False,2020-03-23
2681,Protective effects of hypericin against infectious bronchitis virus induced apoptosis and reactive oxygen species in chicken embryo kidney cells,"ABSTRACT Avian infectious bronchitis virus (IBV), a coronavirus, causes infectious bronchitis leading to enormous economic loss in the poultry industry worldwide. Hypericin (HY) is an excellent compound that has been investigated in antiviral, antineoplastic, and antidepressant. To investigate the inhibition effect of HY on IBV infection in chicken embryo kidney (CEK) cells, 3 different experimental designs: pre-treatment of cells prior to IBV infection, direct treatment of IBV-infected cell...",f48acaa9b87ab31e2be26149983fd243c4a9be6f,custom_license,True,False,2019-12-31
4174,The psychological impact of the COVID-19 epidemic on college students in China,"Abstract A COVID-19 epidemic has been spreading in China and other parts of the world since December 2019. The epidemic has brought not only the risk of death from infection but also unbearable psychological pressure. We sampled college students from Changzhi medical college by using cluster sampling. They responded to a questionnaire packet that included the 7-item Generalized Anxiety Disorder Scale (GAD-7) and those inquiring the participants’ basic information. We received 7,143 responses...",abca4179b3826ec453deacc4aed1030a0ded2700,custom_license,True,False,2020-03-20
...,...,...,...,...,...,...,...
36243,The deadly coronaviruses: The 2003 SARS pandemic and the 2020 novel coronavirus epidemic in China,"Abstract The 2019-nCoV is officially called SARS-CoV-2 and the disease is named COVID-19. This viral epidemic in China has led to the deaths of over 1800 people, mostly elderly or those with an underlying chronic disease or immunosuppressed state. This is the third serious Coronavirus outbreak in less than 20 years, following SARS in 2002–2003 and MERS in 2012.

In [138]:
economic_papers.to_csv("economic_abstracts.csv", index=False)

## Economic seems to be the only keyword, and works in title, not abstract

In [37]:
return_papers(economic_keywords, covid_papers, 0)

,title,abstract,sha,full_text_file,has_pdf_parse,has_pmc_xml_parse
12071,"Global epidemiology of coronavirus disease 2019 (COVID-19): disease incidence, daily cumulative index, mortality, and their association with country healthcare resources and economic status","ABSTRACT It has been 2 months since the first case of coronavirus disease 2019 (COVID-19) was reported in Wuhan, China. So far, COVID-19 has affected 85 403 patients in 57 countries/territories and has caused 2924 deaths in 9 countries. However, epidemiological data differ between countries. Although China had higher morbidity and mortality than other sites, the number of new daily cases in China has been lower than outside of China since 26 February 2020. The incidence ranged from 61.44 per...",dc1e6aa9ceaf4d2129fa5c16f2649e4688405832,custom_license,True,False
22116,Estimates and determinants of economic impacts from influenza‐like illnesses caused by respiratory viruses in Australian children attending childcare: a cohort study,"BACKGROUND: Influenza and other respiratory infections cause excess winter morbidity in children. This study assessed the economic impact of influenza‐like illness (ILI) on families with children attending childcare using a societal perspective. METHODS: We conducted a prospective cohort study in 90 childcare centres and one general practitioner clinics in Sydney, Australia, during 2010. Healthy children aged ≥6 months to <3 years were enrolled. Economic impacts of ILI (temperature ≥37·8°C o...",b3fd1c9139c31dc46f7b65873260adcb21f47192,custom_license,True,True
30629,"Seroprevalence of economically important viral pathogens in swine populations of Trinidad and Tobago, West Indies","The objective of this study was to evaluate the seroprevalence and identify the strains of swine influenza virus (SwIV), as well as the seroprevalence of porcine parvovirus (PPV), transmissible gastroenteritis virus (TGEV), porcine reproductive and respiratory syndrome virus (PRRSV), porcine respiratory coronavirus (PRCV), porcine circovirus type 2 (PCV-2), and classical swine fever virus (CSFV) in pigs in Trinidad and Tobago (T&T). Blood samples (309) were randomly collected from pigs at fa...",479307c95d4d31ef10f0c813e00261b72d9c755e,custom_license,True,True
37298,"A reliable, practical, and economical protocol for inducing diarrhea and severe dehydration in the neonatal calf.","Fifteen healthy, colostrum-fed, male dairy calves, aged 2 to 7 d were used in a study to develop a diarrhea protocol for neonatal calves that is reliable, practical, and economical. After instrumentation and recording baseline data, diarrhea and dehydration were induced by administering milk replacer [16.5 mL/kg of body weight (BW), PO], sucrose (2 g/kg in a 20% aqueous solution, p.o.), spironolactone and hydrochlorothiazide (1 mg/kg, PO) every 8 h, and furosemide (2 mg/kg, i.m., q6h). Calve...",NaN,NaN,False,False
41989,Impacts of social and economic factors on the transmission of coronavirus disease (COVID-19) in China,"This paper examines the role of various socioeconomic factors in mediating the local and cross-city transmissions of the novel coronavirus 2019 (COVID-19) in China. We implement a machine learning approach to select instrumental variables that strongly predict virus transmission among the rich exogenous weather characteristics. Our 2SLS estimates show that the stringent quarantine, massive lockdown and other public health measures imposed in late January significantly reduced the transmissio...",de055f09fef2776bc78bac5d58c4131301b2025f,biorxiv_medrxiv,True,False
42970,Economic Impacts of Wuhan 2019-nCoV on China and the World,"Uncertainties over the Wuhan 2019 Novel Coronavirus (2019-nCoV), which has killed 1,017 people and sickened more than 43,100 as of Feb 11,(1) has interrupted global trade and supply chains, depressed asset prices, and forced multinational businesses to make hard decisions with limited information. This article is protected by copyright. All right

## Corona may also be too broad, but not sure how to limit otherwise

## Search full text files

In [139]:
def find_keyword(keywords, text):
    """
    Iterates through a list of keywords and searches them in a string of text.

    inputs:
      keywords: list of keywords
      text: string of text

    output: number of times keywords are found in the text
    """
    find = []
    for keyword in keywords:
        find.extend(re.findall(keyword, text.lower()))
    return len(find)

In [140]:
def search_body_text(sha, folder1, folder2, keywords, sentence_only):
    """
    Searches a single full length text for sentences/paragraphs which contain a list of keywords.

    inputs:
      sha: sha file name
      folder1: text folder name
      folder2: pdf or pmc folder name
      keywords: list of keywords to search for
      sentence_only: whether or not to show sentence only or full paragraph
    
    output: list of sentences/paragraphs found containing keywords
    """

    #open text file
    with open('./CORD-19-research-challenge/'+folder1+'/'+folder1+'/'+folder2+'/'+sha+'.json') as f:
        file = json.load(f)
    
    found = []
    for text_dict in file["body_text"]:
        
        #if show_sentence_only, then split the paragraph into sentences, then look for keywords
        if sentence_only:
            sentences = text_dict["text"].split(". ")
            for sentence in sentences:
                count = find_keyword(keywords, sentence)
                if count > 0:
                    found.append(sentence)
                    
        #otherwise, show the whole paragraph
        else:
            count = find_keyword(keywords, text_dict["text"])
            if count > 0:
                #print(text_dict["section"])
                found.append(text_dict["text"])
                
    return(found)

In [141]:
def automated_lit_search(metadata_subset, keywords, sentence_only=True):
    """
    Creates a table keyword findings.
    
    inputs:
      metadata_subset: subset of metadata file to search
      keywords: list of keywords to search
      sentence_only: whether or not to show sentence only or full paragraph
    
    output: dataframe table of results with columns containing index, title, and text snippet
    """
    results = []
    
    indices = metadata_subset[metadata_subset['has_pdf_parse'] == True].index
    indices_pmc = metadata_subset[metadata_subset['has_pmc_xml_parse'] == True].index
    indices.append(indices_pmc)
    
    for index in indices:
        
        #find text location
        sha = metadata_subset["sha"][index].split(';')[0]
        folder1 = metadata_subset["full_text_file"][index]
        if metadata_subset['has_pdf_parse'][index] == True:
            folder2 = 'pdf_json'
        elif metadata_subset['has_pmc_xml_parse'][index] == True:
            folder2 = 'pmc_json'
        
        #open text and search for keywords
        found = search_body_text(sha, folder1, folder2, keywords, sentence_only)
        if len(found) > 0:
            for f in found:
                results.append([index, metadata_subset["title"][index], f])
                
    results_df = pd.DataFrame(results, columns=["index","title","text"])
    return(results_df)

In [142]:
intervention_sentences = automated_lit_search(intervention_papers, intervention_keywords, True)
intervention_sentences.to_csv('intervention_sentences.csv', index=False)

In [144]:
intervention_paragraphs = automated_lit_search(intervention_papers, intervention_keywords, False)
intervention_paragraphs.to_csv('intervention_paragraphs.csv', index=False)

In [145]:
economic_sentences = automated_lit_search(economic_papers, economic_keywords, True)
economic_sentences.to_csv('economic_sentences.csv', index=False)

In [146]:
economic_paragraphs = automated_lit_search(economic_papers, economic_keywords, False)
economic_paragraphs.to_csv('economic_paragraphs.csv', index=False)